# **Manifold Learning**

In [1]:
# 경고 메시지 무시 설정
import warnings
warnings.filterwarnings('ignore')  # 경고 메시지가 출력되지 않도록 설정

# 데이터 시각화를 위한 라이브러리
import matplotlib.pyplot as plt  # 그래프와 시각화를 위한 라이브러리

# 수학적 계산 및 데이터 처리를 위한 라이브러리
import math  # 수학 함수 사용을 위한 라이브러리
import numpy as np  # 행렬 및 다차원 배열 계산을 위한 라이브러리
import pandas as pd  # 데이터 분석과 처리를 위한 라이브러리 (DataFrame 지원)

# MATLAB 파일을 읽고 데이터를 처리하기 위한 라이브러리
import scipy.io  # MATLAB 파일 입출력을 지원하는 SciPy의 서브모듈

# 데이터 차원 축소를 위한 라이브러리
from sklearn.decomposition import PCA  # 주성분 분석(PCA)을 위한 모듈
from sklearn import manifold  # 비선형 차원 축소 및 임베딩 기법을 제공하는 모듈


In [ ]:
# Pandas DataFrame 출력 옵션 설정
pd.options.display.max_columns = 7  # 출력 시 최대 7개의 열만 표시

# MATLAB 형식의 데이터를 로드
mat = scipy.io.loadmat('../data/face_data.mat')  # MATLAB 파일에서 데이터 로드
df = pd.DataFrame(mat['images']).T  # 'images' 키의 데이터를 DataFrame으로 변환 후 전치(transpose)

# 데이터의 이미지 개수 및 픽셀 수 계산
num_images, num_pixels = df.shape  # 이미지 수와 각 이미지의 총 픽셀 수 계산
pixels_per_dimension = int(math.sqrt(num_pixels))  # 이미지의 한 변의 픽셀 수 계산 (정사각형 가정)

# 이미지를 회전하여 픽셀 배열을 수정
for idx in df.index:  # DataFrame의 각 행(이미지)에 대해 반복
    df.loc[idx] = df.loc[idx].values.reshape(pixels_per_dimension,  # 1D 픽셀 배열을 2D 배열로 변환
                                             pixels_per_dimension).T.reshape(-1)  # 회전 후 다시 1D 배열로 변환

# 첫 5개의 행 출력
print(df.head())  # 변환된 데이터의 첫 5개 행 출력
print("")
print("df shape :", df.shape)  # 데이터프레임의 크기 출력

In [ ]:
# DataFrame을 리스트 형태로 변환
list_df = np.array(df.values.tolist())  # DataFrame의 값을 numpy 배열로 변환

# 2x3의 서브플롯 생성
fig, ax = plt.subplots(2, 3, subplot_kw=dict(xticks=[], yticks=[]))  # 2행 3열의 서브플롯 생성 (축 눈금 비활성화)

# 이미지 시각화
for i, axi in enumerate(ax.flat):  # 서브플롯의 각 축에 대해 반복
    axi.imshow(list_df[100 * i].reshape(64, 64), cmap='gray_r')  # 1D 배열 이미지를 64x64 형태로 변환 후 표시 (회색조 반전)


In [4]:
def manifold_plot(name, origin_df, manifold_2D):
    """
    2D 임베딩 결과를 시각화하고, 원본 이미지를 일부 플롯 위에 표시하는 함수.

    Parameters:
    - name (str): 임베딩 기법의 이름 (예: 'PCA', 't-SNE').
    - origin_df (DataFrame): 원본 이미지 데이터가 포함된 DataFrame.
    - manifold_2D (DataFrame): 2D 임베딩 결과 데이터프레임, 'Component 1', 'Component 2' 열 포함.
    """

    # 플롯 설정
    fig = plt.figure()  # 새로운 Figure 생성
    fig.set_size_inches(10, 10)  # 플롯 크기 설정
    ax = fig.add_subplot(111)  # 단일 서브플롯 추가
    ax.set_title('2D Components from %s of Facial Images' % name)  # 플롯 제목 설정
    ax.set_xlabel('Component: 1')  # x축 레이블 설정
    ax.set_ylabel('Component: 2')  # y축 레이블 설정

    # 플롯 위에 40개의 이미지를 표시
    x_size = (max(manifold_2D['Component 1']) - min(manifold_2D['Component 1'])) * 0.08  # 이미지 x축 크기 설정
    y_size = (max(manifold_2D['Component 2']) - min(manifold_2D['Component 2'])) * 0.08  # 이미지 y축 크기 설정
    for i in range(40):
        img_num = np.random.randint(0, num_images)  # 랜덤하게 이미지를 선택
        x0 = manifold_2D.loc[img_num, 'Component 1'] - (x_size / 2.)  # 이미지 왼쪽 경계
        y0 = manifold_2D.loc[img_num, 'Component 2'] - (y_size / 2.)  # 이미지 아래쪽 경계
        x1 = manifold_2D.loc[img_num, 'Component 1'] + (x_size / 2.)  # 이미지 오른쪽 경계
        y1 = manifold_2D.loc[img_num, 'Component 2'] + (y_size / 2.)  # 이미지 위쪽 경계

        # 원본 이미지를 2D 임베딩 위에 표시
        img = origin_df.iloc[img_num, :].values.reshape(pixels_per_dimension, pixels_per_dimension)  # 이미지 재구성
        ax.imshow(img, aspect='auto', cmap=plt.cm.gray,  # 이미지 색상과 보간(interpolation) 설정
                  interpolation='nearest', zorder=100000, extent=(x0, x1, y0, y1))

    # 2D 임베딩 데이터 산점도 표시
    ax.scatter(manifold_2D['Component 1'], manifold_2D['Component 2'], marker='.', alpha=0.7)  # 산점도 추가

    # 축 레이블을 포즈 정보로 설정
    ax.set_ylabel('Up-Down Pose')  # y축 레이블 업데이트
    ax.set_xlabel('Right-Left Pose')  # x축 레이블 업데이트

    # 플롯 표시
    plt.show()

## **1. PCA 결과 확인**

In [ ]:
# PCA 모델 초기화 및 학습
pca = PCA(n_components=2)  # 주성분 분석(PCA) 객체를 생성, 2개의 주성분으로 축소 설정
pca.fit(df)  # DataFrame(df)을 기반으로 PCA를 학습하여 주요 축(주성분) 2개를 찾음

# 데이터를 새로운 주성분 공간으로 변환
pca_2D = pca.transform(df)  # PCA를 적용하여 데이터를 2D 공간으로 투영
pca_2D = pd.DataFrame(pca_2D, columns=['Component 1', 'Component 2'])  # 결과를 DataFrame으로 변환

# 변환된 데이터의 상위 5개 행 출력
pca_2D.head()  # 'Component 1'과 'Component 2'로 구성된 데이터의 상위 5개 행을 출력


In [ ]:
manifold_plot("PCA", df, pca_2D)

***

## **2. ISOMAP(n=5) 결과 확인**

In [ ]:
# Isomap 모델 초기화 및 학습
iso = manifold.Isomap(n_neighbors=5, n_components=2)  # Isomap 객체 생성, 5개의 이웃 사용, 2차원으로 축소 설정
iso.fit(df)  # 데이터 df를 기반으로 Isomap 학습 수행

# 데이터를 새로운 2D 임베딩 공간으로 변환
isomap_2Da = iso.transform(df)  # 학습된 Isomap 모델로 데이터를 2D 공간으로 투영
isomap_2D = pd.DataFrame(isomap_2Da, columns=['Component 1', 'Component 2'])  # 결과를 DataFrame으로 변환

# 변환된 데이터의 상위 5개 행 출력
isomap_2D.head()  # 'Component 1'과 'Component 2'로 구성된 데이터의 상위 5개 행을 출력


In [ ]:
manifold_plot("ISOMAP(n=5)", df, isomap_2D)

***

## **3. LLE(n=5) 결과 확인**

In [ ]:
# Locally Linear Embedding (LLE) 모델 초기화 및 학습
LLE = manifold.LocallyLinearEmbedding(n_neighbors=5, n_components=2)  # LLE 객체 생성, 5개의 이웃 사용, 2차원으로 축소 설정
LLE.fit(df)  # 원본 데이터 df를 기반으로 LLE 학습 수행

# 데이터를 새로운 2D 임베딩 공간으로 변환
LLE_manifold_2D_ = LLE.transform(df)  # 학습된 LLE 모델로 데이터를 2D 공간으로 투영
LLE_manifold_2D = pd.DataFrame(LLE_manifold_2D_, columns=['Component 1', 'Component 2'])  # 결과를 DataFrame으로 변환

# 변환된 데이터의 상위 5개 행 출력
LLE_manifold_2D.head()  # 'Component 1'과 'Component 2'로 구성된 데이터의 상위 5개 행을 출력


In [ ]:
manifold_plot("LLE", df, LLE_manifold_2D)

***

## **4. TSNE 결과 확인**

In [ ]:
# t-SNE 모델 초기화 및 학습
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)  
# t-SNE 객체 생성
# - n_components=2: 결과를 2차원으로 축소
# - init='pca': 초기값으로 PCA를 사용
# - random_state=0: 재현성을 위해 난수 시드 설정

tsne.fit(df)  # 원본 데이터 df를 기반으로 t-SNE 학습 수행

# 데이터를 새로운 2D 임베딩 공간으로 변환
tsne_2Da = tsne.fit_transform(df)  # 학습된 t-SNE 모델로 데이터를 2D 공간으로 투영
tsne_2D = pd.DataFrame(tsne_2Da, columns=['Component 1', 'Component 2'])  # 결과를 DataFrame으로 변환

# 변환된 데이터의 상위 5개 행 출력
tsne_2D.head()  # 'Component 1'과 'Component 2'로 구성된 데이터의 상위 5개 행을 출력


In [ ]:
manifold_plot("TSNE", df, tsne_2D)